In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [36]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()


In [37]:

# データの前処理
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0

1.0
0.0


In [4]:
# One-Hot-encoding
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_o = enc.fit_transform(y_train[:, np.newaxis])
y_test_o = enc.fit_transform(y_test[:, np.newaxis])
print(y_train.shape)
print(y_train_o.shape)

(60000,)
(60000, 10)


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_o, test_size=0.2)
print(X_train.shape)
print(X_val.shape)

(48000, 784)
(12000, 784)


# 【問題1】全結合層のクラス化¶
全結合層のクラス化を行なってください。 以下に雛形を載せました。コンストラクタで重みやバイアスの初期化をして、あとはフォワードとバックワードのメソッドを用意します。重みW、バイアスB、およびフォワード時の入力Xをインスタンス変数として保持しておくことで、煩雑な入出力は不要になります。

なお、インスタンスも引数として渡すことができます。そのため、初期化方法のインスタンスinitializerをコンストラクタで受け取れば、それにより初期化が行われます。渡すインスタンスを変えれば、初期化方法が変えられます。

また、引数として自身のインスタンスselfを渡すこともできます。これを利用してself.optimizer.update(self)という風に層の重みの更新が可能です。更新に必要な値は複数ありますが、全て全結合層が持つインスタンス変数にすることができます。

初期化方法と最適化手法のクラスについては後述します



In [6]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.W = initializer.W(self.n_nodes1, self.n_nodes2)
        self.B =  initializer.B(self.n_nodes2)
        # adagrad用
        #if Adagrad = True: 
        self.h_W = np.zeros((n_nodes1, n_nodes2))
        self.h_B = np.zeros(n_nodes2)

    
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """      
        
        self.A =  (X @ self.W) + self.B
        return self.A
    
    def backward(self, dA, Z):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        self.Z = Z
        self.dB = np.sum(dA, axis=0)
        self.dW = self.Z.T @ dA 
        dZ = dA @ self.W.T
        # 更新
        self = self.optimizer.update(self)
        return dZ

# 【問題2】初期化方法のクラス化¶
初期化を行うコードをクラス化してください。

前述のように、全結合層のコンストラクタに初期化方法のインスタンスを渡せるようにします。以下の雛形に必要なコードを書き加えていってください。標準偏差の値（sigma）はコンストラクタで受け取るようにすることで、全結合層のクラス内にこの値（sigma）を渡さなくてすむようになります。

これまで扱ってきた初期化方法はSimpleInitializerクラスと名付けることにします。

In [7]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
      
    """
    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :　重みの初期値
        """
        self.W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return self.W
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :　バイアスの初期値
        """
        self.B = self.sigma * np.random.randn(n_nodes2)
        return self.B

# 【問題3】最適化手法のクラス化
最適化手法のクラス化を行なってください。

最適化手法に関しても初期化方法同様に全結合層にインスタンスとして渡します。バックワードのときにself.optimizer.update(self)のように更新できるようにします。以下の雛形に必要なコードを書き加えていってください。

これまで扱ってきた最適化手法はSGDクラス（Stochastic Gradient Descent、確率的勾配降下法）として作成します。

In [8]:
class SGD():
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr, batch):
        self.lr = lr
        self.batch = batch
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        layer.W = layer.W - self.lr * layer.dW / self.batch
        layer.B = layer.B - self.lr * layer.dB / self.batch
        return layer

# 【問題4】活性化関数のクラス化
活性化関数のクラス化を行なってください。

ソフトマックス関数のバックプロパゲーションには交差エントロピー誤差の計算も含む実装を行うことで計算が簡略化されます。

In [9]:
class Softmax():
    def forward(self, A):
        # overflow対策
        A_max = np.max(A, axis=1)
        exp_A = np.exp(A - A_max.reshape(-1, 1))
        sum_exp_A = np.sum(exp_A, axis=1).reshape(-1, 1)
        return exp_A / sum_exp_A
    
    def backward(self, Z, Y):
        loss = self._cross_entropy(Z, Y)
        D  = Z - Y
        return D, loss 
    
    def _cross_entropy(self,Z, y):
        """
        パラメータ
        ーーーーーーーーーーーーー
        y : 正解ラベル（one-hot表現）
        z３　：　クラスの確率
        n : バッチサイズ

        """
        if y.ndim == 1:    # 次元が 1 の場合
            y = y.reshape(1, y.size)
        batch_size = y.shape[0]
        return -1* np.sum(y * np.log(Z)) / batch_size

# 【問題5】ReLUクラスの作成¶
現在一般的に使われている活性化関数であるReLU（Rectified Linear Unit）をReLUクラスとして実装してください

In [10]:
class Relu():
    def forward(self, A):
        self.mask = (A <= 0)
        a = A.copy()
        self.A = A
        a[self.mask] = 0
        return np.maximum(0, A)
      
    def backward(self, dA):
        dA[self.mask] = 0
        out = dA
        return out

# 【問題6】重みの初期値
ここまでは重みやバイアスの初期値は単純にガウス分布で、標準偏差をハイパーパラメータとして扱ってきました。しかし、どのような値にすると良いかが知られています。シグモイド関数やハイパボリックタンジェント関数のときは Xavierの初期値 （またはGlorotの初期値）、ReLUのときは Heの初期値 が使われます。

### XavierInitializerクラス

In [11]:
class XavierInitializer():
    """
    Xavierによる初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
      
    """
    def __init__(self, sigma):
        self.sigma = 1/ np.sqrt(sigma)
        
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :　重みの初期値
        """
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :　バイアスの初期値
        """
        B = self.sigma * np.random.randn(n_nodes2)
        return B

### HeInitializerクラス

In [12]:

class HeInitializer():
    
    def __init__(self, sigma):
        self.sigma = np.sqrt(2 / sigma)
        
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :　重みの初期値
        """
        self.W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return self.W
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :　バイアスの初期値
        """
        self.B = self.sigma * np.random.randn(n_nodes2)
        return self.B

# 【問題7】最適化手法
学習率は学習過程で変化させていく方法が一般的です。基本的な手法である AdaGrad のクラスを作成してください。

In [13]:
class AdaGrad:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr, batch):
        self.lr = lr
        self.batch = batch
        # 過去の勾配の２乗和の保管用
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """

        layer.h_W += (layer.dW/ self.batch) * (layer.dW/ self.batch)
        layer.h_B += (layer.dB/self.batch) * (layer.dB / self.batch)
        layer.W -= self.lr * (1 / (np.sqrt(layer.h_W) + 1e-07)) * (layer.dW/ self.batch)
        layer.B -= self.lr * (1 / (np.sqrt(layer.h_B) + 1e-07)) * (layer.dB/ self.batch)

        return layer

# 【問題8】クラスの完成¶
任意の構成で学習と推定が行えるScratchDeepNeuralNetrowkClassifierクラスを完成させてください。

In [14]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=10):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [15]:
class ScratchDeepNeuralNetrowkClassifier():
    
    def __init__(self, lr=0.01,sigma=0.1, n_iter=30, batch=20, vervose=False):
        self.iter = n_iter
        self.lr = lr
        self.sigma = sigma
        self.batch = batch
        self.loss = np.zeros(self.iter)
        self.val_loss = np.zeros(self.iter)
        
    
    def fit(self, X, y,  X_val=None, y_val=None):
        self.n_features = 784
        self.n_nodes1 = 400
        self.n_nodes2 =  200
        self.n_output = 10
        
        #更新方法の定義
        self.optimizer = AdaGrad(self.lr, self.batch)
        # 各層の定義
        self.FC1 = FC(self.n_features, self.n_nodes1, HeInitializer(self.n_features), self.optimizer)
        self.activation1 = Relu()
        self.FC2 = FC(self.n_nodes1, self.n_nodes2, HeInitializer(self.n_nodes1), self.optimizer)
        self.activation2 = Relu()
        self.FC3 = FC(self.n_nodes2, self.n_output, HeInitializer(self.n_nodes2), self.optimizer)
        self.activation3 = Softmax()
        
        # 学習開始
        for i in range(self.iter):
            # １エポックのloss記録保管
            history = np.zeros(X.shape[0] // self.batch)
            val_history = np.zeros(X_val.shape[0] // self.batch)
            #  バッチサイズのカウント
            count = 0
            #　ミニバッチの取り出しコード
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.batch)
            for mini_X_train, mini_y_train in get_mini_batch:
                # forward
                A1 = self.FC1.forward(mini_X_train)
                Z1 = self.activation1.forward(A1)
                A2 = self.FC2.forward(Z1)
                Z2 = self.activation2.forward(A2)
                A3 = self.FC3.forward(Z2)
                Z3 = self.activation3.forward(A3)

                # backward
                dA3, history[count] = self.activation3.backward(Z3, mini_y_train)
                dZ2 = self.FC3.backward(dA3, Z2)
                dA2 = self.activation2.backward(dZ2)
                dZ1 = self.FC2.backward(dA2, Z1)
                dA1 = self.activation1.backward(dZ1)
                dZ0 = self.FC1.backward(dA1, mini_X_train)
                count += 1
            
            A1 = self.FC1.forward(X)
            Z1 = self.activation1.forward(A1)
            A2 = self.FC2.forward(Z1)
            Z2 = self.activation2.forward(A2)
            A3 = self.FC3.forward(Z2)
            Z3 = self.activation3.forward(A3)
            
            _, history= self.activation3.backward(Z3, y)
            self.loss[i] = history
            
            # X_valあった場合の処理
            if np.any(X_val): 
                A1 = self.FC1.forward(X_val)
                Z1 = self.activation1.forward(A1)
                A2 = self.FC2.forward(Z1)
                Z2 = self.activation2.forward(A2)
                A3 = self.FC3.forward(Z2)
                Z3 = self.activation3.forward(A3)
                # 学習記録
                _, val_history = self.activation3.backward(Z3, y_val)
                self.val_loss[i] = val_history
            
            
    # 推定
    def predict(self, X):
        """
        ニューラルネットワーク分類器を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            推定結果
        """
        A1 = self.FC1.forward(X)
        Z1 = self.activation1.forward(A1)
        A2 = self.FC2.forward(Z1)
        Z2 = self.activation2.forward(A2)
        A3 = self.FC3.forward(Z2)
        Z3 = self.activation3.forward(A3)
        return np.argmax(Z3, axis= 1)

# 【問題9】学習と推定
層の数や活性化関数を変えたいくつかのネットワークを作成してください。そして、MNISTのデータを学習・推定し、Accuracyを計算してください。#

In [16]:
from sklearn.metrics import accuracy_score

In [17]:
SDNNC = ScratchDeepNeuralNetrowkClassifier()

In [18]:

SDNNC.fit(X_train, y_train, X_val, y_val)

In [26]:
plt.plot(SDNNC.loss)
plt.plot(SDNNC.val_loss)
plt.show();

In [27]:
y_pred = SDNNC.predict(X_test)
accuracy_score(y_test, y_pred)

0.9813

# 追加
別モデル

In [28]:
class ScratchDeepNeuralNetrowkClassifier1():
    
    def __init__(self, lr=0.01,sigma=0.1, n_iter=30, batch=20, vervose=False):
        self.iter = n_iter
        self.lr = lr
        self.sigma = sigma
        self.batch = batch
        self.loss = np.zeros(self.iter)
        self.val_loss = np.zeros(self.iter)
        
    
    def fit(self, X, y,  X_val=None, y_val=None):
        self.n_features = 784
        self.n_nodes1 = 400
        self.n_nodes2 =  200
        self.n_nodes3 = 100
        self.n_output = 10
        
        #更新方法の定義
        self.optimizer = SGD(self.lr, self.batch)
        # 各層の定義
        self.FC1 = FC(self.n_features, self.n_nodes1, HeInitializer(self.n_features), self.optimizer)
        self.activation1 = Relu()
        self.FC2 = FC(self.n_nodes1, self.n_nodes2, HeInitializer(self.n_nodes1), self.optimizer)
        self.activation2 = Relu()
        self.FC3 = FC(self.n_nodes2, self.n_nodes3, HeInitializer(self.n_nodes2), self.optimizer)
        self.activation3 = Relu()
        self.FC4 = FC(self.n_nodes3, self.n_output, HeInitializer(self.n_nodes3), self.optimizer)
        self.activation4 = Softmax()
        
        # 学習開始
        for i in range(self.iter):
            # １エポックのloss記録保管
            history = np.zeros(X.shape[0] // self.batch)
            val_history = np.zeros(X_val.shape[0] // self.batch)
            #  バッチサイズのカウント
            count = 0
            #　ミニバッチの取り出しコード
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.batch)
            for mini_X_train, mini_y_train in get_mini_batch:
                # forward
                A1 = self.FC1.forward(mini_X_train)
                Z1 = self.activation1.forward(A1)
                A2 = self.FC2.forward(Z1)
                Z2 = self.activation2.forward(A2)
                A3 = self.FC3.forward(Z2)
                Z3 = self.activation3.forward(A3)
                A4 = self.FC4.forward(Z3)
                Z4 = self.activation4.forward(A4)

                # backward
                dA4, history[count] = self.activation4.backward(Z4, mini_y_train)
                dZ3 = self.FC4.backward(dA4, Z3)
                dA3 = self.activation3.backward(dZ3)
                dZ2 = self.FC3.backward(dA3, Z2)
                dA2 = self.activation2.backward(dZ2)
                dZ1 = self.FC2.backward(dA2, Z1)
                dA1 = self.activation1.backward(dZ1)
                dZ0 = self.FC1.backward(dA1, mini_X_train)
                count += 1
            
            A1 = self.FC1.forward(mini_X_train)
            Z1 = self.activation1.forward(A1)
            A2 = self.FC2.forward(Z1)
            Z2 = self.activation2.forward(A2)
            A3 = self.FC3.forward(Z2)
            Z3 = self.activation3.forward(A3)
            A4 = self.FC4.forward(Z3)
            Z4 = self.activation4.forward(A4)
            
            _, history= self.activation3.backward(Z3, y)
            self.loss[i] = history
            
            # X_valあった場合の処理
            if np.any(X_val): 
                A1 = self.FC1.forward(X_val)
                Z1 = self.activation1.forward(A1)
                A2 = self.FC2.forward(Z1)
                Z2 = self.activation2.forward(A2)
                A3 = self.FC3.forward(Z2)
                Z3 = self.activation3.forward(A3)
                A4 = self.FC4.forward(Z3)
                Z4 = self.activation4.forward(A4)
                # 学習記録
                _, val_history = self.activation4.backward(Z4, y_val)
                self.val_loss[i] = val_history
            
            
    # 推定
    def predict(self, X):
        """
        ニューラルネットワーク分類器を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            推定結果
        """
        A1 = self.FC1.forward(X)
        Z1 = self.activation1.forward(A1)
        A2 = self.FC2.forward(Z1)
        Z2 = self.activation2.forward(A2)
        A3 = self.FC3.forward(Z2)
        Z3 = self.activation3.forward(A3)
        A4 = self.FC4.forward(Z3)
        Z4 = self.activation4.forward(A4)
        return np.argmax(Z3, axis= 1)

In [29]:
SDNNC1 = ScratchDeepNeuralNetrowkClassifier()

In [30]:
SDNNC.fit(X_train, y_train, X_val, y_val)

In [40]:
plt.plot(SDNNC.loss)
plt.plot(SDNNC.val_loss)
plt.show();

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
y_pred = SDNNC.predict(X_test)
accuracy_score(y_test, y_pred)

0.9813